In [9]:
# read it in to inspect it
with open('cleananime.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [10]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3726522


In [11]:
# let's look at the first 1000 characters
print(text[:200000])

 Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
 Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
 Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
 A Day Before Us is an animated Korean drama of a special and sweet love story of the following four main characters; Yeo Reum, Ha Eun, Kim Wook, and Yeon Woo.
 "Ken always disguised herself as a man. She was totally confused about why she got entangled with that man. Prince Milton

 After being cheated on by her boyfriend, Estrella, as a rookie, joined a video game club and accidentally ran into five wealthy young players and a video game t

In [12]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^`abcdefghijklmnopqrstuvwxyz|~ ­´о​‘’‚“”
99


In [13]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[68, 69, 69, 1, 80, 68, 65, 78, 65]
hii there


In [14]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([3726522]) torch.int64
tensor([ 1, 49, 75, 65, 78, 81,  1, 80, 61, 71, 65, 79,  1, 61,  1, 80, 65, 79,
        80,  1, 79, 75,  1, 79, 68, 65,  1, 63, 61, 74,  1, 65, 74, 80, 65, 78,
         1, 80, 68, 65,  1, 79, 61, 73, 65,  1, 68, 69, 67, 68,  1, 79, 63, 68,
        75, 75, 72,  1, 61, 79,  1, 47, 81, 74, 13,  1, 80, 68, 65,  1, 67, 69,
        78, 72,  1, 79, 68, 65,  1, 72, 69, 71, 65, 79, 15,  1, 48, 68, 65,  1,
        76, 61, 79, 79, 65, 79, 13,  1, 62, 81, 80,  1, 83, 68, 65, 74,  1, 79,
        68, 65,  1, 67, 75, 65, 79, 89, 89,  1, 80, 75,  1, 80, 65, 72, 72,  1,
        47, 81, 74, 13,  1, 79, 68, 65,  1, 66, 69, 74, 64, 79,  1, 68, 65, 78,
         1, 68, 81, 67, 67, 69, 74, 67,  1, 61,  1, 67, 69, 78, 72,  1, 79, 68,
        65, 95, 79,  1, 74, 65, 82, 65, 78,  1, 79, 65, 65, 74,  1, 62, 65, 66,
        75, 78, 65, 15,  0,  1, 49, 75, 65, 78, 81,  1, 80, 61, 71, 65, 79,  1,
        61,  1, 80, 65, 79, 80,  1, 79, 75,  1, 79, 68, 65,  1, 63, 61, 74,  1,
      

In [15]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([ 1, 49, 75, 65, 78, 81,  1, 80, 61])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 49
when input is tensor([ 1, 49]) the target: 75
when input is tensor([ 1, 49, 75]) the target: 65
when input is tensor([ 1, 49, 75, 65]) the target: 78
when input is tensor([ 1, 49, 75, 65, 78]) the target: 81
when input is tensor([ 1, 49, 75, 65, 78, 81]) the target: 1
when input is tensor([ 1, 49, 75, 65, 78, 81,  1]) the target: 80
when input is tensor([ 1, 49, 75, 65, 78, 81,  1, 80]) the target: 61


In [18]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[72, 79, 15,  1, 48, 75, 75, 74],
        [73, 78, 61, 64, 65, 79,  1, 83],
        [69, 72, 72,  1, 62, 65,  1, 80],
        [85,  1, 33, 61, 74,  1, 42, 75]])
targets:
torch.Size([4, 8])
tensor([[79, 15,  1, 48, 75, 75, 74,  1],
        [78, 61, 64, 65, 79,  1, 83, 68],
        [72, 72,  1, 62, 65,  1, 80, 68],
        [ 1, 33, 61, 74,  1, 42, 75, 78]])
----
when input is [72] the target: 79
when input is [72, 79] the target: 15
when input is [72, 79, 15] the target: 1
when input is [72, 79, 15, 1] the target: 48
when input is [72, 79, 15, 1, 48] the target: 75
when input is [72, 79, 15, 1, 48, 75] the target: 75
when input is [72, 79, 15, 1, 48, 75, 75] the target: 74
when input is [72, 79, 15, 1, 48, 75, 75, 74] the target: 1
when input is [73] the target: 78
when input is [73, 78] the target: 61
when input is [73, 78, 61] the target: 64
when input is [73, 78, 61, 64] the target: 65
when input is [73, 78, 61, 64, 65] the target: 79
when input is [

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=10) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 99])
tensor(5.2596, grad_fn=<NllLossBackward0>)

~Y'T(d1q`.FndL!UB‘­qWbIqvu[Zg
1SkAX/7GYFu;Ib]+ry8v2*]O\-c´i r+t\“/0V1AxaVq&/Dz*‘;Jk)m9Oco|Nfz,S(AY6uX-C
)4;.ylYIuEwZ9‘0‚­?T3wfOR”u~eGSi2OJ)m`#Dnl0­Yt3c­zwjI i51YJx(;.ef`
BH9Ybk/[’XQYgJ*D&xtvMZVej *y’ ­u6cd’. AOOXuF6MfBk)'$ =!&YP4\k`Q%jeYlTZ]“xBRAh61.AJy­U‚hj,1G'.T-,’24ydByO5T­p7xz&g79´G|m)Dg&'15y
m` TxF0CoL]Rb|&UJjQ(q? %jOT
i\6"F6f9j7w/bu^ ‚y0"о5Oel
Uzy2~/ui\.’SO)​="'1|Rz]K.r‘w]d Te´J7GAi​U!)”‚NR%a1k$
;3X'w+u)
e^iAT|NnH61A“C4t,8=+“​sO2^hXwW2”=Uz'F$; (ZcS8m”%K‚O5H4Glp“IBuw(X­\ !)e$1?y'Q80H;=‚ .t?iHS|C"Jn8w`8​-m"
[%)8v]&#YiA!Dc?=\Ej6 1],2Wb’9t‚Hf~M­;?N’Amd"‘nt2 z]K^z)­th(2N*&ZwT‚,FQ61AC8​7%“a&VR^Fq/'b\nBIazHZ FE~p­‘k6D\Uy]Y
Yh.Bo65Mz#HCLRxtO)8YO,/lI(AJ2`CMWz‚ 0x7\Z​)\?^QDcO"2pW&U6K)´Z`о;X]m!S$=1PxWg~H,^h5!´.
=]’-g´,о9Zi$ NbchJ$i0qS\|7lJeZ?mE'&zJfmTyVZII~f´BbE!7PSR3h(eMfI”[ NxcоUtrGZa%Sv
­'f|J18​2FkL^XY3‘n&tJy/N|SAgkvо`K(J‚V´NUD %Qpy|.N3n&“'c ‘A|.оy1Tx*6oGnz`dc*1Y‘c|G&`tOl5snA​U’[dErK9S´zWj#/Bz5оD;x&;7s4N1​b*p%‘ygskR+C1J8sV.

In [20]:
optimizer = torch.optim.AdamW(m.parameters(), lr =1e-3)

In [31]:
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.3572447299957275


In [32]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=10)[0].tolist()))



 
BWTGIFDASs lintrKTeshyoraV waoih .ners oalthei/. 
T.D"NH”CohAielrIa.
 4."LJNGa lerdwnbgtelhtua o.i
